In [74]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error, r2_score
from xgboost import XGBRegressor

# 데이터 로드

In [49]:
filepath = "/Users/seoyeongbubu/Documents/Github_sy/jump-to-github/merged_sum.csv"

In [50]:
df = pd.read_csv(filepath)

In [51]:
df

,구,연도,강남구,강동구,강북구,강서구,관악구,광진구,구로구,금천구,...,증감률,1인가구_비율,20세 미만,65세 이상,0~14세,소비자물가,저소득노인_65~79비율,저소득노인_80이상비율,기초생활수급자총인원,기초생활수급자비율
0,강남구,2017,1,0,0,0,0,0,0,0,...,1.3,12.36,539,7607,68591,97.645,4.64,6.81,1706,22.43
1,강남구,2018,1,0,0,0,0,0,0,0,...,-0.1,12.00,421,7834,66657,99.086,45.66,36.18,1840,23.49
2,강남구,2019,1,0,0,0,0,0,0,0,...,5.3,12.98,607,8393,66767,99.466,1.68,4.44,2959,35.26
3,강남구,2020,1,0,0,0,0,0,0,0,...,4.4,13.58,498,9162,65904,100.000,8.87,7.07,3994,43.59
4,강남구,2021,1,0,0,0,0,0,0,0,...,4.3,14.31,397,10292,64520,102.500,2.19,5.48,3970,38.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,중랑구,2019,0,0,0,0,0,0,0,0,...,6.1,13.51,201,10920,37243,99.466,5.54,20.87,4826,44.19
171,중랑구,2020,0,0,0,0,0,0,0,0,...,8.4,14.68,206,12190,35444,100.000,4.07,16.39,6275,51.48
172,중랑구,2021,0,0,0,0,0,0,0,0,...,6.9,15.90,169,13358,33325,102.500,9.10,31.97,7074,52.96
173,중랑구,2022,0,0,0,0,0,0,0,0,...,6.4,16.97,213,14569,32188,107.720,2.93,7.34,7778,53.39


In [52]:
df.shape

(175, 48)

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 48 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   구              175 non-null    object 
 1   연도             175 non-null    int64  
 2   강남구            175 non-null    int64  
 3   강동구            175 non-null    int64  
 4   강북구            175 non-null    int64  
 5   강서구            175 non-null    int64  
 6   관악구            175 non-null    int64  
 7   광진구            175 non-null    int64  
 8   구로구            175 non-null    int64  
 9   금천구            175 non-null    int64  
 10  노원구            175 non-null    int64  
 11  도봉구            175 non-null    int64  
 12  동대문구           175 non-null    int64  
 13  동작구            175 non-null    int64  
 14  마포구            175 non-null    int64  
 15  서대문구           175 non-null    int64  
 16  서초구            175 non-null    int64  
 17  성동구            175 non-null    int64  
 18  성북구       

# target / feature

In [54]:
df.columns

Index(['구', '연도', '강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구',
       '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구',
       '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구', '단독주택_계', '아파트_계',
       '연립주택_계', '다세대주택_계', '비주거용 건물내 주택_계', '값', '노령화지수', '총인구', '총인구_남자',
       '총인구_여자', '전년 대비 증감', '증감률', '1인가구_비율', '20세 미만', '65세 이상', '0~14세',
       '소비자물가', '저소득노인_65~79비율', '저소득노인_80이상비율', '기초생활수급자총인원', '기초생활수급자비율'],
      dtype='object')

In [55]:
# 원핫 인코딩 컬럼
region_cols = [
    "강남구","강동구","강북구","강서구","관악구","광진구","구로구","금천구",
    "노원구","도봉구","동대문구","동작구","마포구","서대문구","서초구","성동구",
    "성북구","송파구","양천구","영등포구","용산구","은평구","종로구","중구","중랑구"
]

In [56]:
# 기존 수치형 컬럼
numeric_cols = [
    "연도", "노령화지수", "총인구", "1인가구_비율",
    "65세 이상", "0~14세", "소비자물가",
    "저소득노인_65~79비율", "저소득노인_80이상비율",
    "기초생활수급자비율"
]

In [57]:
# 타깃
target = "값"

# lag (전 연도 고독사 사망 인원수 값)

In [58]:
df = df.sort_values(["구","연도"])
df["lag_1"] = df.groupby("구")[target].shift(1) # 작년
df["lag_2"] = df.groupby("구")[target].shift(2) # 재작년

# rolling mean (3년 이동평균 값)

In [59]:
# "구"별 target으로 데이터를 묶고
# apply : 한 번씩 lambda 함수 적용시킴
# lambda x : x.rolling(3).mean() : groupby로 묶인 값들의 과거 3년치 평균을 계산함
roll = df.groupby("구")[target].apply(lambda x : x.rolling(3).mean())

In [60]:
# 인덱스 재 정렬
roll = roll.reset_index(level=0, drop=True)

In [61]:
# 데이터 프레임의 컬럼으로 추가시키기
df["roll_3"] = roll

# 조합 변수 (인구 * 노령화), (노인비율 * 저소득 비율)

### -> 인구와 노령화지수가 같이 오를 때 위험도가 더 오르는 패턴을 잡기 위해 컬럼을 조합한 변수를 적용한다.

In [62]:
df["인구x노령화"] = df["총인구"] * df["노령화지수"]
df["노인비x저소득"] = df["65세 이상"] * df["저소득노인_80이상비율"]

# 결측치 제거

In [63]:
df = df.dropna()
print(df.shape)

(125, 53)


# Feature 재 설정

In [64]:
# v.0.4에 추가 한 기능의 컬럼명 먼저 설정
addition_cols = ["lag_1","lag_2","roll_3","인구x노령화","노인비x저소득"]

In [67]:
feature_cols = numeric_cols + region_cols + addition_cols

X = df[feature_cols]
y = df[target]

# 학습/테스트 세트 분리

In [68]:
X_train, X_test, y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=42)

# 전처리

In [69]:
# 스케일링할 수치형 컬럼 설정
scale_cols = [c for c in feature_cols if c not in region_cols]

In [72]:
preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), scale_cols),
        ("cat", "passthrough", region_cols)
    ]
)

In [75]:
pipeline = Pipeline(steps=[("preprocess", preprocess)])

# 스케일 한 데이터 학습시키기

In [76]:
X_train_scaled = pipeline.fit_transform(X_train)
X_test_scaled = pipeline.transform(X_test)

# 다중공선성 VIF

In [77]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [79]:
# 스케일 된 데이터들을 DataFrame형태로 다시 만들기
scaled_df = pd.DataFrame(X_train_scaled, columns=scale_cols + region_cols)

In [81]:
# VIF 진행
vif_data = []

for i, col in enumerate(scaled_df.columns):
    vif = variance_inflation_factor(scaled_df.values, i)
    vif_data.append([col, vif])

vif_df = pd.DataFrame(vif_data, columns=["feature", "VIF"])

In [82]:
# VIF 결과
print(vif_df.sort_values("VIF", ascending=False))

          feature          VIF
2             총인구  5979.034600
5           0~14세  2902.906741
12         roll_3   383.693000
19            관악구   356.930812
13         인구x노령화   277.905490
32            송파구   255.505620
38             중구   234.028856
37            종로구   196.694997
3         1인가구_비율   196.518577
10          lag_1   192.032242
4          65세 이상   172.725713
29            서초구   157.160564
1           노령화지수   149.409480
35            용산구   115.603491
18            강서구   113.804832
0              연도    97.943232
25           동대문구    92.682572
14        노인비x저소득    79.556460
8    저소득노인_80이상비율    76.051263
11          lag_2    70.031785
15            강남구    59.271924
23            노원구    58.066249
22            금천구    53.516443
6           소비자물가    53.438191
36            은평구    49.856193
33            양천구    36.413350
28           서대문구    33.638560
39            중랑구    33.005991
17            강북구    31.416742
24            도봉구    30.728703
31            성북구    30.392089
16      